# Nutrition Health Survey- Age Prediction
### Hackathon: 2 - Summer Analytics 2025, IIT Guwahati
* NIDHISH VANDEKAR 
* v.nidhish@iitg.ac.in

In [45]:
# Importing Required Libraries
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score

In [3]:
# Getting Data ready 
train_df = pd.read_csv("Train_Data.csv")
test_df = pd.read_csv("Test_Data.csv")

In [4]:
# Dropping identifier column
train_df.drop("SEQN", axis=1, inplace=True)
test_ids = test_df["SEQN"]
test_df.drop("SEQN", axis=1, inplace=True)

In [5]:
train_df.head(2)

,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,Adult
1,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,Adult


In [6]:
test_df.head(2)

,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,34.0,1.0,1.0,32.2,96.0,2.0,135.0,15.11
1,12.0,2.0,2.0,26.3,100.0,2.0,141.0,15.26


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1966 entries, 0 to 1965
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   RIDAGEYR   1957 non-null   float64
 1   RIAGENDR   1948 non-null   float64
 2   PAQ605     1953 non-null   float64
 3   BMXBMI     1948 non-null   float64
 4   LBXGLU     1953 non-null   float64
 5   DIQ010     1948 non-null   float64
 6   LBXGLT     1955 non-null   float64
 7   LBXIN      1957 non-null   float64
 8   age_group  1952 non-null   object 
dtypes: float64(8), object(1)
memory usage: 138.4+ KB


In [8]:
# Checking and filling null values 
train_df.isnull().sum()

RIDAGEYR      9
RIAGENDR     18
PAQ605       13
BMXBMI       18
LBXGLU       13
DIQ010       18
LBXGLT       11
LBXIN         9
age_group    14
dtype: int64

In [22]:
# filling catogerical columns with mode values
diq_mode = train_df["DIQ010"].mode()[0]
train_df["DIQ010"].fillna(diq_mode, inplace=True)

gender_mode = train_df["RIAGENDR"].mode()[0]
train_df["RIAGENDR"].fillna(gender_mode, inplace=True)

# Replace confusing codes in PAQ605 with NaN
train_df.loc[train_df["PAQ605"].isin([7, 9]), "PAQ605"] = np.nan

# Fill missing values in PAQ605 with mode value
paq_mode = train_df["PAQ605"].mode()[0]
train_df["PAQ605"].fillna(paq_mode, inplace=True)

/var/folders/wj/k7shrh55397gz734l220hvcm0000gn/T/ipykernel_28696/1069532005.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["DIQ010"].fillna(diq_mode, inplace=True)
/var/folders/wj/k7shrh55397gz734l220hvcm0000gn/T/ipykernel_28696/1069532005.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

In [26]:
# Filling rest of the values with mean values
train_df.fillna(train_df.mean(numeric_only=True), inplace=True)

In [30]:
# Remove rows with missing age_group values
train_df.dropna(inplace=True)

In [31]:
train_df.isnull().sum()

RIDAGEYR     0
RIAGENDR     0
PAQ605       0
BMXBMI       0
LBXGLU       0
DIQ010       0
LBXGLT       0
LBXIN        0
age_group    0
dtype: int64

In [32]:
test_df.isnull().sum()

RIDAGEYR    3
RIAGENDR    2
PAQ605      1
BMXBMI      1
LBXGLU      1
DIQ010      1
LBXGLT      2
LBXIN       1
dtype: int64

In [39]:
# filling catogerical columns in test_df with mode values 
gen_mode_test = test_df["RIAGENDR"].mode()[0]
test_df["RIAGENDR"].fillna(gen_mode_test, inplace=True)

diq_mode_test = test_df["DIQ010"].mode()[0]
test_df["DIQ010"].fillna(diq_mode_test, inplace=True)

# Replace confusing codes in PAQ605 with NaN
test_df.loc[test_df["PAQ605"].isin([7, 9]), "PAQ605"] = np.nan

# Fill missing values in PAQ605 with mode value
paq_mode_test = test_df["PAQ605"].mode()[0]
test_df["PAQ605"].fillna(paq_mode_test, inplace=True)

/var/folders/wj/k7shrh55397gz734l220hvcm0000gn/T/ipykernel_28696/1641720345.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df["RIAGENDR"].fillna(gen_mode_test, inplace=True)
/var/folders/wj/k7shrh55397gz734l220hvcm0000gn/T/ipykernel_28696/1641720345.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [42]:
# Filling rest of the values with mean values
test_df.fillna(test_df.mean(numeric_only=True), inplace=True)

In [43]:
test_df.isnull().sum()

RIDAGEYR    0
RIAGENDR    0
PAQ605      0
BMXBMI      0
LBXGLU      0
DIQ010      0
LBXGLT      0
LBXIN       0
dtype: int64

In [46]:
# numerical encoding age_group column 
le = LabelEncoder()
train_df["age_group"] = le.fit_transform(train_df["age_group"])

In [56]:
# set up random seed 
np.random.seed(42)

#separate features and targets 
x = train_df.drop("age_group", axis=1)
y = train_df["age_group"]

# Splitting into train test split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [58]:
# Feature scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(test_df)

In [60]:
# Instentiate model
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [71]:
from sklearn.metrics import f1_score, classification_report

y_val_pred = clf.predict(x_val)
print("F1 Score:", f1_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

F1 Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       328
           1       1.00      1.00      1.00        63

    accuracy                           1.00       391
   macro avg       1.00      1.00      1.00       391
weighted avg       1.00      1.00      1.00       391



In [72]:
test_preds = clf.predict(x_test)

In [74]:
np.unique(test_preds) 

array([0, 1])

In [75]:
# Create DataFrame with just the predictions
submission = pd.DataFrame({"age_group": test_preds})

# Save to CSV
submission.to_csv("submission.csv", index=False)